In [1]:
from data_loader import load_image_data

X_train, X_val, y_train, y_val, label_map = load_image_data(
    csv_file="../data/image_data_rel.csv",
    img_size=(299, 299),
    images_per_label=20,
    validation_split=0.2
)